In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
boston = datasets.load_boston()
X = boston.data
y = boston.target
X = X[y < 50.0]
y = y[y < 50.0]

In [3]:
X.shape

(490, 13)

In [4]:
import sys
sys.path.append('/home/zhuf/CodingSpace/JupyterNotebook/machine_learning')
from playML.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

# 线性回归的解析解

要求解的参数$\theta$，第一个元素是截距，后面是X各个特征对应的系数

$X_b$是在X上扩展一列全为1元素后得到的矩阵

$$\theta = (X_b^TX_b)^{−1}X_b^Ty$$

预测值：

$$\hat y = X_b\theta$$

- 上述解称为正规方程解(Normal Equation)
- 线性回归算法能够得到最终的解析解，这在机器学习算法中是比较少见的，一般机器学习算法很难得到最终解析解
- 可以直接使用numpy提供的矩阵运算获得最终结果，但是实际上这是比较低效的计算方法，尤其是当计算规模较大的时候（无论是行数多还是列数多）
- 缺点：正规方程解的计算复杂度达到了$O(n^3)$,即使进行计算优化，也有$O(n^{2.4})$的复杂度
- 优点：是不需要对数据做归一化处理
- Sklearn其实是使用梯度下降算法进行求解的

# 使用自己的线性回归模型

In [6]:
from playML.LinearRegression import  LinearRegression

In [7]:
reg = LinearRegression()

In [8]:
reg.fit_normal(X_train, y_train)

LinearRegression_MyOwnVersion

In [10]:
reg.score(X_test, y_test)

0.8129794056212918

In [11]:
reg.intercept_

34.1173997232074

In [12]:
reg.coef_

array([-1.20354261e-01,  3.64423279e-02, -3.61493155e-02,  5.12978140e-02,
       -1.15775825e+01,  3.42740062e+00, -2.32311760e-02, -1.19487594e+00,
        2.60101728e-01, -1.40219119e-02, -8.35430488e-01,  7.80472852e-03,
       -3.80923751e-01])

# Sklearn中的线性回归

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
lin_reg = LinearRegression()

In [15]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
lin_reg.score(X_test, y_test)

0.812979405621281

In [17]:
lin_reg.intercept_

34.11739972322955

In [18]:
lin_reg.coef_

array([-1.20354261e-01,  3.64423279e-02, -3.61493155e-02,  5.12978140e-02,
       -1.15775825e+01,  3.42740062e+00, -2.32311760e-02, -1.19487594e+00,
        2.60101728e-01, -1.40219119e-02, -8.35430488e-01,  7.80472852e-03,
       -3.80923751e-01])